In [ ]:

import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:

soybean_large = fetch_ucirepo(id=90)

# data (as pandas dataframes)
X = soybean_large.data.features
y = soybean_large.data.targets


In [ ]:

# Substituir '?' por NaN
X = X.replace('?', np.nan)

# Preencher valores ausentes com o valor mais frequente de cada coluna (moda)
for col in X.columns:
    X[col].fillna(X[col].mode()[0], inplace=True)


In [ ]:

encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = encoder.fit_transform(X)


In [ ]:

le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stop], verbose=1)


In [ ]:

y_pred = model.predict(X_test).argmax(axis=1)
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))
